In [26]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


In [17]:
github_token = os.environ.get("GITHUB_TOKEN")
print("github_token", github_token)

github_token None


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=7) cached=False logprobs=None thought=None


In [30]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

In [31]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Plan me a great sunny vacation", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[]
source='assistant' models_usage=RequestUsage(prompt_tokens=21, completion_tokens=671) content="Sure! Here’s a detailed plan for a sunny vacation to a beautiful destination. Let’s go with **Queenstown, New Zealand**, a fantastic spot known for its stunning scenery and outdoor activities. Here’s a week-long itinerary:\n\n### Day 1: Arrival in Queenstown\n- **Morning**: Arrive at Queenstown Airport and take a scenic shuttle ride to your accommodation.\n- **Accommodation**: Check into a lakeside hotel or cozy Airbnb with mountain views.\n- **Afternoon**: Stroll around Lake Wakatipu and explore the vibrant Queenstown Gardens.\n- **Evening**: Enjoy dinner at a lakeside restaurant, trying local cuisine like lamb or fresh seafood.\n\n### Day 2: Adventure Day\n- **Morning**: Start your day with a thrilling **jet boat ride** on the Shotover River.\n- **Afternoon**: Take the **Skyline Gondola** to Bob's Peak for panoramic views and lunch at the restaurant at the top.\n- **Evening**: Try a fun 